# 青空文庫データクレンジング

In [1]:
import os
import pandas as pd
from pathlib import Path

## データ取得

- 以降は，Google Colaboで実行されることを想定しています．
- 参考：[Pythonで青空文庫データを自然言語処理向けにさくっと一括テキスト整形＋前処理](https://qiita.com/dzbt_dzbt/items/593dbd698a07c12a771c)

In [2]:
folder_name = 'aozora'

In [3]:
!cd /content
!rm -rf aozora # 必要に応じて削除
!mkdir aozora #フォルダ作成
%cd aozora

/content/aozora


- データ量が多いので，夏目漱石のテキストだけを取得するように設定
- これまでのコミット履歴は不要なので，最新のものだけ取得している(--depth=1)
  - 参考：[【GitHub記録】特定のディレクトリのみcloneする方法](https://itosae.com/github_sparsechekout/)

In [4]:
!git init
!git remote add origin https://github.com/aozorabunko/aozorabunko.git
!git config core.sparsecheckout true
!echo 'cards/000148/' > .git/info/sparse-checkout # 000148:夏目漱石
!git pull --depth=1 origin master

Initialized empty Git repository in /content/aozora/.git/
remote: Enumerating objects: 74477, done.
remote: Counting objects: 100% (74477/74477), done.
remote: Compressing objects: 100% (60121/60121), done.
remote: Total 74477 (delta 24900), reused 53950 (delta 14169), pack-reused 0
Receiving objects: 100% (74477/74477), 2.25 GiB | 31.09 MiB/s, done.
Resolving deltas: 100% (24900/24900), done.
From https://github.com/aozorabunko/aozorabunko
 * branch                master     -> FETCH_HEAD
 * [new branch]          master     -> origin/master


カレントフォルダを元にもとしておく．
- 参考:https://qiita.com/bear_montblanc/items/64d7efd9e4ea7caa98f9

In [7]:
# カレントフォルダを元に戻しておく
%cd /content/
!ls

/content
aozora	sample_data


## 関数定義

In [8]:
author_id = '000148'  # 青空文庫の作家番号
author_name = '夏目漱石'  # 青空文庫の表記での作家名

write_title = False # 2カラム目に作品名を入れるか
write_header = True # 1行目をカラム名にするか（カラム名「text」「title」）
save_utf8_org = True  # 元データをUTF-8にしたテキストファイルを保存するか
split_kuten = True # 句点「。」で分割するか

out_dir = Path(f'/content/aozora/cards/{author_id}/files/')  # ファイル出力先
tx_org_dir = Path(out_dir / './org/')  # 元テキストのUTF-8変換ファイルの保存先
tx_edit_dir = Path(out_dir / './edit/')  # テキスト整形後のファイル保存先

In [9]:
def text_cleanse_df(df):
    # 本文の先頭を探す（'---…'区切りの直後から本文が始まる前提）
    head_tx = list(df[df['text'].str.contains(
        '-------------------------------------------------------')].index)
    # 本文の末尾を探す（'底本：'の直前に本文が終わる前提）
    atx = list(df[df['text'].str.contains('底本：')].index)
    if head_tx == []:
        # もし'---…'区切りが無い場合は、作家名の直後に本文が始まる前提
        head_tx = list(df[df['text'].str.contains(author_name)].index)
        head_tx_num = head_tx[0]+1
    else:
        # 2個目の'---…'区切り直後から本文が始まる
        head_tx_num = head_tx[1]+1
    df_e = df[head_tx_num:atx[0]]

    # 句点で分割
    if split_kuten:
        # df.assign(コメント=df['コメント'].str.split(r'(?<=。)(?=..)')).explode('コメント')
        df_e = df_e.assign(text=df['text'].str.split(r'(?<=。)(?=..)')).explode('text')

    # 青空文庫の書式削除
    df_e = df_e.replace({'text': {'《.*?》': ''}}, regex=True)
    df_e = df_e.replace({'text': {'［.*?］': ''}}, regex=True)
    df_e = df_e.replace({'text': {'｜': ''}}, regex=True)

    # 字下げ（行頭の全角スペース）を削除
    df_e = df_e.replace({'text': {'　': ''}}, regex=True)

    # 節区切りを削除
    df_e = df_e.replace({'text': {'^.$': ''}}, regex=True)
    df_e = df_e.replace({'text': {'^―――.*$': ''}}, regex=True)
    df_e = df_e.replace({'text': {'^＊＊＊.*$': ''}}, regex=True)
    df_e = df_e.replace({'text': {'^×××.*$': ''}}, regex=True)

    # 記号、および記号削除によって残ったカッコを削除
    # df_e = df_e.replace({'text': {'―': ''}}, regex=True)
    # df_e = df_e.replace({'text': {'…': ''}}, regex=True)
    # df_e = df_e.replace({'text': {'※': ''}}, regex=True)
    df_e = df_e.replace({'text': {'「」': ''}}, regex=True)

    # 一文字以下で構成されている行を削除
    df_e['length'] = df_e['text'].map(lambda x: len(x))
    df_e = df_e[df_e['length'] > 1]

    # インデックスがずれるので振りなおす
    df_e = df_e.reset_index().drop(['index'], axis=1)

    # 空白行を削除する（念のため）
    df_e = df_e[~(df_e['text'] == '')]

    # インデックスがずれるので振り直し、文字の長さの列を削除する
    df_e = df_e.reset_index().drop(['index', 'length'], axis=1)
    return df_e

In [10]:
def save_cleanse_text(target_file):
    try:
        # ファイルの読み込み
        print(target_file)
        # Pandas DataFrameとして読み込む（cp932で読み込まないと異体字が読めない）
        df_tmp = pd.read_csv(target_file, encoding='cp932', names=['text'])
        # 元データをUTF-8に変換してテキストファイルを保存
        if save_utf8_org:
            out_org_file_nm = Path(target_file.stem + '_org_utf-8.tsv')
            df_tmp.to_csv(Path(tx_org_dir / out_org_file_nm), sep='\t',
                          encoding='utf-8', index=None)
        # テキスト整形
        df_tmp_e = text_cleanse_df(df_tmp)
        if write_title:
            # タイトル列を作る
            df_tmp_e['title'] = df_tmp['text'][0]
        out_edit_file_nm = Path(target_file.stem + '_clns_utf-8.txt')
        df_tmp_e.to_csv(Path(tx_edit_dir / out_edit_file_nm), sep='\t',
                        encoding='utf-8', index=None, header=write_header)
    except:
        print(f'ERROR: {target_file}')

In [11]:
def main():
    # zipファイルのリストを作成
    zip_list = list(out_dir.glob('*.zip'))
    # 保存ディレクトリを作成しておく
    tx_edit_dir.mkdir(exist_ok=True, parents=True)
    if save_utf8_org:
        tx_org_dir.mkdir(exist_ok=True, parents=True)

    for target_file in zip_list:
        save_cleanse_text(target_file)

## データ取得＆クレンジング実行

In [12]:
main()

/content/aozora/cards/000148/files/4681_ruby_9461.zip
/content/aozora/cards/000148/files/2671_ruby_6335.zip
/content/aozora/cards/000148/files/1746_ruby_18324.zip
/content/aozora/cards/000148/files/59017_ruby_70128.zip
/content/aozora/cards/000148/files/755_ruby_3617.zip
ERROR: /content/aozora/cards/000148/files/755_ruby_3617.zip
/content/aozora/cards/000148/files/753_ruby_1701.zip
/content/aozora/cards/000148/files/1104_ruby_4554.zip
/content/aozora/cards/000148/files/1749_ruby_19433.zip
/content/aozora/cards/000148/files/779_ruby_2743.zip
ERROR: /content/aozora/cards/000148/files/779_ruby_2743.zip
/content/aozora/cards/000148/files/58277_ruby_69902.zip
/content/aozora/cards/000148/files/4683_ruby_9475.zip
/content/aozora/cards/000148/files/770_ruby.zip
ERROR: /content/aozora/cards/000148/files/770_ruby.zip
/content/aozora/cards/000148/files/780_ruby_569.zip
/content/aozora/cards/000148/files/1076_ruby_4527.zip
ERROR: /content/aozora/cards/000148/files/1076_ruby_4527.zip
/content/aozo

In [13]:
!ls /content/aozora/cards/000148/files/edit/789_ruby_5639*

/content/aozora/cards/000148/files/edit/789_ruby_5639_clns_utf-8.txt


In [14]:
df = pd.read_csv('/content/aozora/cards/000148/files/edit/789_ruby_5639_clns_utf-8.txt')
df

,text
0,吾輩は猫である。
1,名前はまだ無い。
2,どこで生れたかとんと見当がつかぬ。
3,何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
4,吾輩はここで始めて人間というものを見た。
...,...
9054,吾輩は死ぬ。
9055,死んでこの太平を得る。
9056,太平は死ななければ得られぬ。
9057,南無阿弥陀仏南無阿弥陀仏。


## 参考
- https://qiita.com/dzbt_dzbt/items/593dbd698a07c12a771c
- https://itosae.com/github_sparsechekout/
- https://qiita.com/bear_montblanc/items/64d7efd9e4ea7caa98f9